## Bayesian Interval Estimation using Markov Chain Monte Carlo  

In this example we compute a Bayesian credible interval using the Markov Chain Monte Carlo method with the RooStats  `MCMCCalculator` class.

The interval is obtained from the Posterior probability function, which is computed from the Bayes theorem using a 
Markov chain Monte Carlo

$$P( \mu | x ) = \frac { \int L (x | \mu , \nu ) \Pi ( \mu, \nu ) d\nu } { \int \int L (x | \mu , \nu ) \Pi ( \mu, \nu ) d \mu d\nu} $$

The output consists of a set of data points (the MC chain) which is distributed according to the posterior distribution. 
This calculator can work with many parameter of interests and many nuisance parameters.


In [ ]:
%jsroot on

In [ ]:
using namespace RooStats;

In [ ]:
MCMCInterval * interval = nullptr;
MCMCIntervalPlot * plot = nullptr; 
ProposalFunction * proposalFunc = nullptr;
RooAbsPdf * priorPdf = nullptr;
TStopwatch tw;

##### 1. Reading the model (Workspace) from input file

First part is just to get the workspace from the input file and retrieve the model and the data set

In [ ]:
//TString fileName = "CountingModelVaryingB.root";
TString fileName = "HiggsBinModel.root";
TString workspaceName = "w";
TString modelConfigName = "ModelConfig";
TString dataName = "data";
TString integrationType = "";  

In [ ]:
auto file = TFile::Open(fileName);

In [ ]:
auto w =  (RooWorkspace*) file->Get(workspaceName);
w->Print();
auto mc = (RooStats::ModelConfig*) w->obj(modelConfigName);
auto  data = w->data(dataName);
auto poi = (RooRealVar*) mc->GetParametersOfInterest()->first();  // get POI 



In [ ]:
// constraint nuisance parameters to +/- 10 sigma of their best fit
a1 = w->var("a1");
a1->setMin(a1->getVal()-10*a1->getError());
a1->setMax(a1->getVal()+10*a1->getError());

a2 = w->var("a2");
a2->setMin(a2->getVal()-10*a2->getError());
a2->setMax(a1->getVal()+10*a2->getError());

#### 2. Create and configure the `MCMCCalculator` class

In [ ]:
RooStats::MCMCCalculator mcmcCalc(*data,*mc);

##### Set a prior PDF, otherwise by default a uniform prior is used

We create here a Prior probability density function for the parameter of interest. 

In [ ]:
//priorPdf = (RooAbsPdf*) w->factory("EXPR::priorPdf('1./sqrt(s)',s)");
//priorPdf = new RooGenericPdf("priorPdf","priorPdf","1./(s)",RooArgList(*w->var("s")));

If a prior pdf has been created, set it in the calculator. Otherwise, in case no prior is set, a default uniform prior in the parameters is used. 

In [ ]:
if (priorPdf) mcmcCalc.SetPriorPdf(*priorPdf);
// this is needed to normalize the prior pdf
RooAbsReal::defaultIntegratorConfig()->method1D().setLabel("RooAdaptiveGaussKronrodIntegrator1D");

##### Set the interval Confidence Level

In [ ]:
mcmcCalc.SetConfidenceLevel(0.90);

##### Set the type of interval.

One can have : 

- shortest interval
- central interval (equal fraction of probability in the tails) 
- lower limit    
- upper limit  

The distinction between lower/upper and central interval is done by setting the left side tail fraction of probability.

Set the type of interval, default is shortest intervals

In [ ]:
//mcmcCalc.SetLeftSideTailFraction(0.5);  // for central intervals
//mcmcCalc.SetLeftSideTailFraction(0.);   // for upper limits

##### Set proposal function and configure Markov-Chain Monte Carlo method

In [ ]:
proposalFunc = new RooStats::SequentialProposal(0.1);
mcmcCalc.SetProposalFunction(*proposalFunc);

In [ ]:
mcmcCalc.SetNumIters(50000);         // Metropolis-Hastings algorithm iterations
mcmcCalc.SetNumBurnInSteps(1000);       // first N steps to be ignored as burn-in

In [ ]:
// define a restricted range for the parameter of interest
//w->var("s")->setMax(20);
mcmcCalc.SetNumBins(100);

#### 3. Compute the interval

Run the calculator and compute the interval from the posterior. 

In [ ]:
tw.Start();
interval = mcmcCalc.GetInterval();
tw.Print();

In [ ]:
cout << 100*mcmcCalc.ConfidenceLevel() << "% interval is : [" 
    << interval->LowerLimit(*poi) << ", "<< interval->UpperLimit(*poi) <<"] "<<endl;

In [ ]:
plot = new RooStats::MCMCIntervalPlot(*interval);

In [ ]:
plot->Draw("HIST"); gPad->Draw();